In [ ]:
import pandas as pd
from tbdynamics.tools.plotting import plot_model_vs_actual
import nevergrad as ng
from tbdynamics.settings import CM_PATH
# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.camau.calibration.utils import get_bcm, load_targets
from multiprocessing import cpu_count

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "smear_positive_death_rate": 0.3655528915762244,
    "smear_negative_death_rate": 0.027358324164819155,
    "smear_positive_self_recovery": 0.18600338108638945,
    "smear_negative_self_recovery": 0.11333894801537307,
    "screening_scaleup_shape": 0.5,
    "screening_inflection_time": 1993,
    # "time_to_screening_end_asymp": 2.1163556520843936,
    # "acf_sensitivity": 0.7685269379793348,
    # "prop_mixing_same_stratum": 0.6920672992582717,
    "incidence_props_pulmonary": 0.5754287819768528,
    "incidence_props_smear_positive_among_pulmonary": 0.2762796281857033,
    # "early_prop_adjuster": -0.017924441638418186,
    # "late_reactivation_adjuster": 1.1083422207175728,
    # "detection_reduction": 0.5993572222604127,
    # "total_population_dispersion": 3644.236227852164,
    # "notif_dispersion": 88.37092488550051,
    # "latent_dispersion": 7.470896188551709,
}
covid_effects = {"detection_reduction": True, "contact_reduction": False}
bcm = get_bcm(params, covid_effects, homo_mixing=False)

In [ ]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(500)
mle_params = rec.value[1]
mle_params

In [ ]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets(CM_PATH / 'targets.yml')

In [ ]:
model = bcm.model

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [ ]:
derived_df_0['percentage_latent'].plot()

In [ ]:
derived_df_0['percentage_latent_adults'].plot()

In [ ]:
derived_df_0['prevalence_infectious'].plot()

In [ ]:
derived_df_0.loc[2014:2019,[f'total_populationXact3_{act3_stratum}' for act3_stratum in ["trial","control"]]].plot()

In [ ]:
derived_df_0.loc[2014:2019,'acf_detectionXact3_trialXorgan_pulmonary'].plot()

In [ ]:
derived_df_0.loc[2016:2019,'acf_detectionXact3_controlXorgan_pulmonary'].plot()